In [ ]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf
from pyspark.context import SparkContext

In [ ]:
import gcsfs
import pandas as pd 

In [ ]:
json_key = '/Users/og/Desktop/de-final-project-388703-903ec0488aa0.json'


conf = SparkConf() \
    .setMaster('local[*]') \
    .setAppName('demographic_data') \
    .set("spark.jars", "./lib/gcs-connector-hadoop3-2.2.5.jar") \
    .set("spark.hadoop.google.cloud.auth.service.account.enable", "true") \
    .set("spark.hadoop.google.cloud.auth.service.account.json.keyfile", json_key)

In [ ]:
sc = SparkContext(conf=conf)

hadoop_conf = sc._jsc.hadoopConfiguration()

hadoop_conf.set("fs.AbstractFileSystem.gs.impl",  "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")
hadoop_conf.set("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
hadoop_conf.set("fs.gs.auth.service.account.json.keyfile", json_key)
hadoop_conf.set("fs.gs.auth.service.account.enable", "true")

In [ ]:
spark = SparkSession.builder \
    .config(conf=sc.getConf()) \
    .getOrCreate()

In [ ]:
bucket = 'de_final_project_bucket'
base_path = 'data/demographic/city/'
years = range(2016,2022)
dfs = []

for year in years:
    file_path = f'{base_path}{year}_California_city_demographic_data.parquet'
    df = spark.read.parquet(f'gs://{bucket}/{file_path}')
    dfs.append(df)

In [ ]:
meta_df = None 

for df in dfs:
    if meta_df is None:
        meta_df = df 
    else:
        meta_df = meta_df.union(df)

In [ ]:
meta_df.show()

In [ ]:
# df = spark.read.parquet('gs://de_final_project_bucket/data/demographic/city/*_Florida_city_demographic_data.parquet')

In [ ]:
column_count = len(meta_df.columns)
print('Column Count:', column_count)

In [ ]:
row_count = meta_df.count()
print('Row Count:', row_count)